In [ ]:
!git clone https://github.com/camlab-ethz/ConvolutionalNeuralOperator.git

Cloning into 'ConvolutionalNeuralOperator'...
remote: Enumerating objects: 496, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 496 (delta 119), reused 87 (delta 87), pack-reused 318 (from 1)
Receiving objects: 100% (496/496), 15.09 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (219/219), done.


In [ ]:
%cd /content/ConvolutionalNeuralOperator/CNO2d_original_version

/content/ConvolutionalNeuralOperator/CNO2d_original_version


In [ ]:
%ls

CNOModule.py    Error_Distribution.py                    ModelSelectionCNO.py  torch_utils/
debug_tools.py  Error_Distribution_VaryingResolution.py  Problems/             TrainCNO.py
dnnlib/         LICENSE_NVIDIA.txt                       readme.md             training/


In [ ]:
pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 8.8 MB/s eta 0:00:00


In [ ]:
import copy
import json
import os
import sys

import pandas as pd
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [ ]:
training_properties = {
    "learning_rate": 0.001,
    "weight_decay": 1e-6,
    "scheduler_step": 10,
    "scheduler_gamma": 0.98,
    "epochs": 1000,
    "batch_size": 16,
    "exp": 1,                # Do we use L1 or L2 errors? Default: L1
    "training_samples": 256  # How many training samples?
}
model_architecture_ = {

    #Parameters to be chosen with model selection:
    "N_layers": 3,            # Number of (D) & (U) blocks
    "channel_multiplier": 32, # Parameter d_e (how the number of channels changes)
    "N_res": 4,               # Number of (R) blocks in the middle networs.
    "N_res_neck" : 6,         # Number of (R) blocks in the BN

    #Other parameters:
    "in_size": 64,            # Resolution of the computational grid
    "retrain": 4,             # Random seed
    "kernel_size": 3,         # Kernel size.
    "FourierF": 0,            # Number of Fourier Features in the input channels. Default is 0.
    "activation": 'cno_lrelu',# cno_lrelu or cno_lrelu_torch or lrelu or

    #Filter properties:
    "cutoff_den": 2.0001,     # Cutoff parameter.
    "lrelu_upsampling": 2,    # Coefficient N_{\sigma}. Default is 2.
    "half_width_mult": 0.8,   # Coefficient c_h. Default is 1
    "filter_size": 6,         # 2xfilter_size is the number of taps N_{tap}. Default is 6.
    "radial_filter": 0,       # Is the filter radially symmetric? Default is 0 - NO.
}

In [ ]:
folder = "/content/ConvolutionalNeuralOperator/CNO2d_original_version/TrainedModels"

In [ ]:
if not os.path.isdir(folder):
    print("Generated new folder")
    os.mkdir(folder)

Generated new folder


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
writer = SummaryWriter(log_dir=folder) #usage of TensorBoard

learning_rate = training_properties["learning_rate"]
epochs = training_properties["epochs"]
batch_size = training_properties["batch_size"]
weight_decay = training_properties["weight_decay"]
scheduler_step = training_properties["scheduler_step"]
scheduler_gamma = training_properties["scheduler_gamma"]
training_samples = training_properties["training_samples"]
p = training_properties["exp"]

In [ ]:
import torch.nn as nn

import torch
#from debug_tools import *
from training.filtered_networks import LReLu, LReLu_regular, LReLu_torch #Either "filtered LReLU" or regular LReLu
from debug_tools import format_tensor_size

In [ ]:
class CNOBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 in_size,
                 out_size,
                 cutoff_den = 2.0001,
                 conv_kernel = 3,
                 filter_size = 6,
                 lrelu_upsampling = 2,
                 half_width_mult  = 0.8,
                 radial = False,
                 batch_norm = True,
                 activation = 'cno_lrelu'
                 ):
        super(CNOBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.in_size  = in_size
        self.out_size = out_size
        self.conv_kernel = conv_kernel
        self.batch_norm = batch_norm

        #---------- Filter properties -----------
        self.citically_sampled = False #We use w_c = s/2.0001 --> NOT critically sampled

        if cutoff_den == 2.0:
            self.citically_sampled = True
        self.in_cutoff  = self.in_size / cutoff_den
        self.out_cutoff = self.out_size / cutoff_den

        self.in_halfwidth =  half_width_mult*self.in_size - self.in_size / cutoff_den
        self.out_halfwidth = half_width_mult*self.out_size - self.out_size / cutoff_den

        #-----------------------------------------

        # We apply Conv -> BN (optional) -> Activation
        # Up/Downsampling happens inside Activation

        pad = (self.conv_kernel-1)//2
        self.value_convolution = torch.nn.Conv2d(in_channels = self.in_channels, out_channels=self.out_channels,
                                           kernel_size=self.conv_kernel, stride = 1,
                                           padding = pad)
        self.gate_convolution = torch.nn.Conv2d(in_channels = self.in_channels, out_channels=self.out_channels,
                                           kernel_size=self.conv_kernel,
                                           padding = pad)

        if self.batch_norm:
            self.value_batch_norm  = nn.BatchNorm2d(self.out_channels)
            self.gate_batch_norm = nn.BatchNorm2d(self.out_channels)

        if activation == "cno_lrelu":
            self.activation  = LReLu(in_channels           = self.in_channels, #In _channels is not used in these settings
                                     out_channels          = self.out_channels,
                                     in_size               = self.in_size,
                                     out_size              = self.out_size,
                                     in_sampling_rate      = self.in_size,
                                     out_sampling_rate     = self.out_size,
                                     in_cutoff             = self.in_cutoff,
                                     out_cutoff            = self.out_cutoff,
                                     in_half_width         = self.in_halfwidth,
                                     out_half_width        = self.out_halfwidth,
                                     filter_size           = filter_size,
                                     lrelu_upsampling      = lrelu_upsampling,
                                     is_critically_sampled = self.citically_sampled,
                                     use_radial_filters    = False)
        elif activation == "cno_lrelu_torch":
            self.activation = LReLu_torch(in_channels           = self.out_channels, #In _channels is not used in these settings
                                            out_channels          = self.out_channels,
                                            in_size               = self.in_size,
                                            out_size              = self.out_size,
                                            in_sampling_rate      = self.in_size,
                                            out_sampling_rate     = self.out_size)
        elif activation == "lrelu":
            self.activation  = LReLu_regular(in_channels           = self.in_channels, #In _channels is not used in these settings
                                             out_channels          = self.out_channels,
                                             in_size               = self.in_size,
                                             out_size              = self.out_size,
                                             in_sampling_rate      = self.in_size,
                                             out_sampling_rate     = self.out_size)
        else:
            raise ValueError("Please specify different activation function")


    def forward(self, value_x, gate_x):
        value_x = self.convolution(value_x)
        gate_x = self.convolution(gate_x)
        if self.batch_norm:
            value_x = self.value_batch_norm(value_x)
            gate_x = self.gate_batch_norm(gate_x)
        value_x = value_x * torch.sigmoid(gate_x)

        return value_x, gate_x

#------------------------------------------------------------------------------

# Contains CNOBlock -> Convolution -> BN

class LiftProjectBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 in_size,
                 out_size,
                 latent_dim = 64,
                 cutoff_den = 2.0001,
                 conv_kernel = 3,
                 filter_size = 6,
                 lrelu_upsampling = 2,
                 half_width_mult  = 0.8,
                 radial = False,
                 batch_norm = True,
                 activation = 'cno_lrelu'
                 ):
        super(LiftProjectBlock, self).__init__()

        self.inter_CNOBlock = CNOBlock(in_channels = in_channels,
                                    out_channels = latent_dim,
                                    in_size = in_size,
                                    out_size = out_size,
                                    cutoff_den = cutoff_den,
                                    conv_kernel = conv_kernel,
                                    filter_size = filter_size,
                                    lrelu_upsampling = lrelu_upsampling,
                                    half_width_mult  = half_width_mult,
                                    radial = radial,
                                    batch_norm = batch_norm,
                                    activation = activation)

        pad = (conv_kernel-1)//2
        self.value_convolution = torch.nn.Conv2d(in_channels = latent_dim, out_channels=out_channels,
                                           kernel_size=conv_kernel, stride = 1,
                                           padding = pad)
        self.gate_convolution = torch.nn.Conv2d(in_channels = latent_dim, out_channels=out_channels,
                                           kernel_size=conv_kernel, stride = 1,
                                           padding = pad)

        self.batch_norm = batch_norm
        if self.batch_norm:
            self.value_batch_norm  = nn.BatchNorm2d(out_channels)
            self.gate_batch_norm = nn.BatchNorm2d(out_channels)

    def forward(self, value_x, gate_x):
        value_x, gate_x = self.inter_CNOBlock(value_x, gate_x)

        value_x = self.value_convolution(value_x)
        gate_x = self.gate_convolution(gate_x)
        if self.batch_norm:
            value_x = self.batch_norm(value_x)
            gate_x = self.batch_norm(gate_x)
        return value_x, gate_x

#------------------------------------------------------------------------------

# Residual Block containts:
    # Convolution -> BN -> Activation -> Convolution -> BN -> SKIP CONNECTION

class ResidualBlock(nn.Module):
    def __init__(self,
                 channels,
                 size,
                 cutoff_den = 2.0001,
                 conv_kernel = 3,
                 filter_size = 6,
                 lrelu_upsampling = 2,
                 half_width_mult  = 0.8,
                 radial = False,
                 batch_norm = True,
                 activation = 'cno_lrelu'
                 ):
        super(ResidualBlock, self).__init__()

        self.channels = channels
        self.size  = size
        self.conv_kernel = conv_kernel
        self.batch_norm = batch_norm

        #---------- Filter properties -----------
        self.citically_sampled = False #We use w_c = s/2.0001 --> NOT critically sampled

        if cutoff_den == 2.0:
            self.citically_sampled = True
        self.cutoff  = self.size / cutoff_den
        self.halfwidth =  half_width_mult*self.size - self.size / cutoff_den

        #-----------------------------------------

        pad = (self.conv_kernel-1)//2
        self.value_convolution1 = torch.nn.Conv2d(in_channels = self.channels, out_channels=self.channels,
                                           kernel_size=self.conv_kernel, stride = 1,
                                           padding = pad)
        self.value_convolution2 = torch.nn.Conv2d(in_channels = self.channels, out_channels=self.channels,
                                           kernel_size=self.conv_kernel, stride = 1,
                                           padding = pad)
        self.gate_convolution1 = torch.nn.Conv2d(in_channels = self.channels, out_channels=self.channels,
                                           kernel_size=self.conv_kernel, stride = 1,
                                           padding = pad)
        self.gate_convolution2 = torch.nn.Conv2d(in_channels = self.channels, out_channels=self.channels,
                                           kernel_size=self.conv_kernel, stride = 1,
                                           padding = pad)

        if self.batch_norm:
            self.value_batch_norm1  = nn.BatchNorm2d(self.channels)
            self.value_batch_norm2  = nn.BatchNorm2d(self.channels)
            self.gate_batch_norm1  = nn.BatchNorm2d(self.channels)
            self.gate_batch_norm2  = nn.BatchNorm2d(self.channels)

        if activation == "cno_lrelu":

            self.activation  = LReLu(in_channels           = self.channels, #In _channels is not used in these settings
                                     out_channels          = self.channels,
                                     in_size               = self.size,
                                     out_size              = self.size,
                                     in_sampling_rate      = self.size,
                                     out_sampling_rate     = self.size,
                                     in_cutoff             = self.cutoff,
                                     out_cutoff            = self.cutoff,
                                     in_half_width         = self.halfwidth,
                                     out_half_width        = self.halfwidth,
                                     filter_size           = filter_size,
                                     lrelu_upsampling      = lrelu_upsampling,
                                     is_critically_sampled = self.citically_sampled,
                                     use_radial_filters    = False)

        elif activation == "cno_lrelu_torch":
            self.activation = LReLu_torch(in_channels           = self.channels, #In _channels is not used in these settings
                                            out_channels          = self.channels,
                                            in_size               = self.size,
                                            out_size              = self.size,
                                            in_sampling_rate      = self.size,
                                            out_sampling_rate     = self.size)
        elif activation == "lrelu":

            self.activation = LReLu_regular(in_channels           = self.channels, #In _channels is not used in these settings
                                            out_channels          = self.channels,
                                            in_size               = self.size,
                                            out_size              = self.size,
                                            in_sampling_rate      = self.size,
                                            out_sampling_rate     = self.size)
        else:
            raise ValueError("Please specify different activation function")


    def forward(self, value_x, gate_x):
        value_out = self.value_convolution1(value_x)
        gate_out = self.gate_convolution1(gate_x)
        if self.batch_norm:
            value_out = self.value_batch_norm1(value_out)
            gate_out = self.gate_batch_norm1(gate_out)
        value_out = value_out * torch.sigmoid(gate_out)
        value_out = self.value_convolution2(value_out)
        gate_out = self.gate_convolution2(gate_out)
        if self.batch_norm:
            value_out = self.value_batch_norm2(value_out)
            gate_out = self.gate_batch_norm2(gate_out)

        return value_x + value_out, gate_x + gate_out
#------------------------------------------------------------------------------

#CNO NETWORK:
class CNO(nn.Module):
    def __init__(self,
                 in_dim,                    # Number of input channels.
                 in_size,                   # Input spatial size
                 N_layers,                  # Number of (D) or (U) blocks in the network
                 N_res = 1,                 # Number of (R) blocks per level (except the neck)
                 N_res_neck = 6,            # Number of (R) blocks in the neck
                 channel_multiplier = 32,   # How the number of channels evolve?
                 conv_kernel=3,             # Size of all the kernels
                 cutoff_den = 2.0001,       # Filter property 1.
                 filter_size=6,             # Filter property 2.
                 lrelu_upsampling = 2,      # Filter property 3.
                 half_width_mult  = 0.8,    # Filter property 4.
                 radial = False,            # Filter property 5. Is filter radial?
                 batch_norm = True,         # Add BN? We do not add BN in lifting/projection layer
                 out_dim = 1,               # Target dimension
                 out_size = 1,              # If out_size is 1, Then out_size = in_size. Else must be int
                 expand_input = False,      # Start with original in_size, or expand it (pad zeros in the spectrum)
                 latent_lift_proj_dim = 64, # Intermediate latent dimension in the lifting/projection layer
                 add_inv = True,            # Add invariant block (I) after the intermediate connections?
                 activation = 'cno_lrelu'   # Activation function can be 'cno_lrelu' or 'lrelu'
                ):

        super(CNO, self).__init__()

        ###################### Define the parameters & specifications #################################################


        # Number of (D) & (U) Blocks
        self.N_layers = int(N_layers)

        # Input is lifted to the half on channel_multiplier dimension
        self.lift_dim = channel_multiplier//2
        self.out_dim  = out_dim

        #Should we add invariant layers in the decoder?
        self.add_inv = add_inv

        # The growth of the channels : d_e parametee
        self.channel_multiplier = channel_multiplier

        # Is the filter radial? We always use NOT radial
        if radial ==0:
            self.radial = False
        else:
            self.radial = True

        ###################### Define evolution of the number features ################################################

        # How the features in Encoder evolve (number of features)
        self.encoder_features = [self.lift_dim]
        for i in range(self.N_layers):
            self.encoder_features.append(2 ** i *   self.channel_multiplier)

        # How the features in Decoder evolve (number of features)
        self.decoder_features_in = self.encoder_features[1:]
        self.decoder_features_in.reverse()
        self.decoder_features_out = self.encoder_features[:-1]
        self.decoder_features_out.reverse()

        for i in range(1, self.N_layers):
            self.decoder_features_in[i] = 2*self.decoder_features_in[i] #Pad the outputs of the resnets

        self.inv_features = self.decoder_features_in
        self.inv_features.append(self.encoder_features[0] + self.decoder_features_out[-1])

        ###################### Define evolution of sampling rates #####################################################

        if not expand_input:
            latent_size = in_size # No change in in_size
        else:
            down_exponent = 2 ** N_layers
            latent_size = in_size - (in_size % down_exponent) + down_exponent # Jump from 64 to 72, for example

        #Are inputs and outputs of the same size? If not, how should the size of the decoder evolve?
        if out_size == 1:
            latent_size_out = latent_size
        else:
            if not expand_input:
                latent_size_out = out_size # No change in in_size
            else:
                down_exponent = 2 ** N_layers
                latent_size_out = out_size - (out_size % down_exponent) + down_exponent # Jump from 64 to 72, for example

        self.encoder_sizes = []
        self.decoder_sizes = []
        for i in range(self.N_layers + 1):
            self.encoder_sizes.append(latent_size // 2 ** i)
            self.decoder_sizes.append(latent_size_out // 2 ** (self.N_layers - i))



        ###################### Define Projection & Lift ##############################################################

        self.lift = LiftProjectBlock(in_channels  = in_dim,
                                     out_channels = self.encoder_features[0],
                                     in_size      = in_size,
                                     out_size     = self.encoder_sizes[0],
                                     latent_dim   = latent_lift_proj_dim,
                                     cutoff_den   = cutoff_den,
                                     conv_kernel  = conv_kernel,
                                     filter_size  = filter_size,
                                     lrelu_upsampling  = lrelu_upsampling,
                                     half_width_mult   = half_width_mult,
                                     radial            = radial,
                                     batch_norm        = False,
                                     activation = activation)
        _out_size = out_size
        if out_size == 1:
            _out_size = in_size

        self.project = LiftProjectBlock(in_channels  = self.encoder_features[0] + self.decoder_features_out[-1],
                                        out_channels = out_dim,
                                        in_size      = self.decoder_sizes[-1],
                                        out_size     = _out_size,
                                        latent_dim   = latent_lift_proj_dim,
                                        cutoff_den   = cutoff_den,
                                        conv_kernel  = conv_kernel,
                                        filter_size  = filter_size,
                                        lrelu_upsampling  = lrelu_upsampling,
                                        half_width_mult   = half_width_mult,
                                        radial            = radial,
                                        batch_norm        = False,
                                        activation = activation)

        ###################### Define U & D blocks ###################################################################

        self.encoder         = nn.ModuleList([(CNOBlock(in_channels  = self.encoder_features[i],
                                                        out_channels = self.encoder_features[i+1],
                                                        in_size      = self.encoder_sizes[i],
                                                        out_size     = self.encoder_sizes[i+1],
                                                        cutoff_den   = cutoff_den,
                                                        conv_kernel  = conv_kernel,
                                                        filter_size  = filter_size,
                                                        lrelu_upsampling = lrelu_upsampling,
                                                        half_width_mult  = half_width_mult,
                                                        radial = radial,
                                                        batch_norm = batch_norm,
                                                        activation = activation))
                                               for i in range(self.N_layers)])

        # After the ResNets are executed, the sizes of encoder and decoder might not match (if out_size>1)
        # We must ensure that the sizes are the same, by aplying CNO Blocks
        self.ED_expansion     = nn.ModuleList([(CNOBlock(in_channels = self.encoder_features[i],
                                                        out_channels = self.encoder_features[i],
                                                        in_size      = self.encoder_sizes[i],
                                                        out_size     = self.decoder_sizes[self.N_layers - i],
                                                        cutoff_den   = cutoff_den,
                                                        conv_kernel  = conv_kernel,
                                                        filter_size  = filter_size,
                                                        lrelu_upsampling = lrelu_upsampling,
                                                        half_width_mult  = half_width_mult,
                                                        radial = radial,
                                                        batch_norm = batch_norm,
                                                        activation = activation))
                                               for i in range(self.N_layers + 1)])

        self.decoder         = nn.ModuleList([(CNOBlock(in_channels  = self.decoder_features_in[i],
                                                        out_channels = self.decoder_features_out[i],
                                                        in_size      = self.decoder_sizes[i],
                                                        out_size     = self.decoder_sizes[i+1],
                                                        cutoff_den   = cutoff_den,
                                                        conv_kernel  = conv_kernel,
                                                        filter_size  = filter_size,
                                                        lrelu_upsampling = lrelu_upsampling,
                                                        half_width_mult  = half_width_mult,
                                                        radial = radial,
                                                        batch_norm = batch_norm,
                                                        activation = activation))
                                               for i in range(self.N_layers)])


        self.decoder_inv    = nn.ModuleList([(CNOBlock(in_channels  =  self.inv_features[i],
                                                        out_channels = self.inv_features[i],
                                                        in_size      = self.decoder_sizes[i],
                                                        out_size     = self.decoder_sizes[i],
                                                        cutoff_den   = cutoff_den,
                                                        conv_kernel  = conv_kernel,
                                                        filter_size  = filter_size,
                                                        lrelu_upsampling = lrelu_upsampling,
                                                        half_width_mult  = half_width_mult,
                                                        radial = radial,
                                                        batch_norm = batch_norm,
                                                        activation = activation))
                                               for i in range(self.N_layers + 1)])


        ####################### Define ResNets Blocks ################################################################

        # Here, we define ResNet Blocks.
        # We also define the BatchNorm layers applied BEFORE the ResNet blocks

        # Operator UNet:
        # Outputs of the middle networks are patched (or padded) to corresponding sets of feature maps in the decoder

        self.res_nets = []
        self.N_res = int(N_res)
        self.N_res_neck = int(N_res_neck)

        # Define the ResNet blocks & BatchNorm
        for l in range(self.N_layers):
            for i in range(self.N_res):
                self.res_nets.append(ResidualBlock(channels = self.encoder_features[l],
                                                   size     = self.encoder_sizes[l],
                                                   cutoff_den = cutoff_den,
                                                   conv_kernel = conv_kernel,
                                                   filter_size = filter_size,
                                                   lrelu_upsampling = lrelu_upsampling,
                                                   half_width_mult  = half_width_mult,
                                                   radial = radial,
                                                   batch_norm = batch_norm,
                                                   activation = activation))
        for i in range(self.N_res_neck):
            self.res_nets.append(ResidualBlock(channels = self.encoder_features[self.N_layers],
                                               size     = self.encoder_sizes[self.N_layers],
                                               cutoff_den = cutoff_den,
                                               conv_kernel = conv_kernel,
                                               filter_size = filter_size,
                                               lrelu_upsampling = lrelu_upsampling,
                                               half_width_mult  = half_width_mult,
                                               radial = radial,
                                               batch_norm = batch_norm,
                                               activation = activation))

        self.res_nets = torch.nn.Sequential(*self.res_nets)


    def forward(self, x):
        gate_x = x
        value_x = torch.ones_like(x)

        #Execute Lift ---------------------------------------------------------
        value_x = self.lift(value_x, gate_x)
        skip = []

        # Execute Encoder -----------------------------------------------------
        for i in range(self.N_layers):

            #Apply ResNet & save the result
            for j in range(self.N_res):
                value_x, gate_x = self.res_nets[i*self.N_res + j](value_x, gate_x)
            skip.append([value_x, gate_x])

            # Apply (D) block
            value_x, gate_x = self.encoder[i](value_x, gate_x)

        #----------------------------------------------------------------------

        # Apply the deepest ResNet (bottle neck)
        for j in range(self.N_res_neck):
            value_x, gate_x = self.res_nets[-j-1](value_x, gate_)

        # Execute Decoder -----------------------------------------------------
        for i in range(self.N_layers):

            # Apply (I) block (ED_expansion) & cat if needed
            if i == 0:
                value_x, gate_x = self.ED_expansion[self.N_layers - i](value_x, gate_x) #BottleNeck : no cat
            else:
                value_x = torch.cat((value_x, self.ED_expansion[self.N_layers - i](skip[-i][0], skip[-i][1])[0]),1)
                gate_x = torch.cat((gate_x, self.ED_expansion[self.N_layers - i](skip[-i][0], skip[-i][1])[1]),1)

            if self.add_inv:
                value_x, gate_x = self.decoder_inv[i](value_x, gate_)
            # Apply (U) block
            value_x, gate_x = self.decoder[i](value_x, gate_x)
        # Cat & Execute Projetion ---------------------------------------------

        value_x = torch.cat((value_x, self.ED_expansion[0](skip[0][0], skip[0][1])[0]),1)
        gate_x = torch.cat((gate_x, self.ED_expansion[0](skip[0][0], skip[0][1])[1]),1)

        value_x, gate_x = self.project(value_x, gate_x)

        del skip
        del y

        return value_x


    def get_n_params(self):
        pp = 0

        for p in list(self.parameters()):
            nn = 1
            for s in list(p.size()):
                nn = nn * s
            pp += nn
        return pp

    def print_size(self):
        nparams = 0
        nbytes = 0

        for param in self.parameters():
            nparams += param.numel()
            nbytes += param.data.element_size() * param.numel()

        print(f'Total number of model parameters: {nparams} (~{format_tensor_size(nbytes)})')

        return nparams

In [ ]:
import random

import h5py
import numpy as np
import torch
from torch.utils.data import DataLoader

from CNOModule import CNO
from torch.utils.data import Dataset

import scipy

from training.FourierFeatures import FourierFeatures

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
def samples_fft(u):
    return scipy.fft.fft2(u, norm='forward', workers=-1)

def samples_ifft(u_hat):
    return scipy.fft.ifft2(u_hat, norm='forward', workers=-1).real

def downsample(u, N):
    N_old = u.shape[-2]
    freqs = scipy.fft.fftfreq(N_old, d=1/N_old)
    sel = np.logical_and(freqs >= -N/2, freqs <= N/2-1)
    u_hat = samples_fft(u)
    u_hat_down = u_hat[:,:,sel,:][:,:,:,sel]
    u_down = samples_ifft(u_hat_down)
    return u_down

#------------------------------------------------------------------------------

#Load default parameters:

def default_param(network_properties):

    if "channel_multiplier" not in network_properties:
        network_properties["channel_multiplier"] = 32

    if "half_width_mult" not in network_properties:
        network_properties["half_width_mult"] = 1

    if "lrelu_upsampling" not in network_properties:
        network_properties["lrelu_upsampling"] = 2

    if "filter_size" not in network_properties:
        network_properties["filter_size"] = 6

    if "out_size" not in network_properties:
        network_properties["out_size"] = 1

    if "radial" not in network_properties:
        network_properties["radial_filter"] = 0

    if "cutoff_den" not in network_properties:
        network_properties["cutoff_den"] = 2.0001

    if "FourierF" not in network_properties:
        network_properties["FourierF"] = 0

    if "retrain" not in network_properties:
        network_properties["retrain"] = 4

    if "kernel_size" not in network_properties:
        network_properties["kernel_size"] = 3

    if "activation" not in network_properties:
        network_properties["activation"] = 'cno_lrelu'

    return network_properties

In [ ]:
class ShearLayerDataset(Dataset):
    def __init__(self, which="training", nf=0, training_samples = 750, s=64, in_dist = True):

        self.s = s
        self.in_dist = in_dist
        #The file:

        if in_dist:
            if self.s==64:
                self.file_data = "/content/drive/MyDrive/PDE Datasets - 2/data/NavierStokes_64x64_IN.h5" #In-distribution file 64x64
            else:
                self.file_data = "/content/drive/MyDrive/PDE Datasets - 2/data/NavierStokes_128x128_IN.h5"   #In-distribution file 128x128
        else:
            self.file_data = "/content/drive/MyDrive/PDE Datasets - 2/data/NavierStokes_128x128_OUT.h5"  #Out-of_-distribution file 128x128

        self.reader = h5py.File(self.file_data, 'r')
        self.N_max = 1024

        self.n_val  = 128
        self.n_test = 128
        self.min_data = 1.4307903051376343
        self.max_data = -1.4307903051376343
        self.min_model = 2.0603253841400146
        self.max_model= -2.0383243560791016

        if which == "training":
            self.length = training_samples
            self.start = 0
        elif which == "validation":
            self.length = self.n_val
            self.start = self.N_max - self.n_val - self.n_test
        elif which == "test":
            self.length = self.n_test
            self.start = self.N_max  - self.n_test

        #Fourier modes (Default is 0):
        self.N_Fourier_F = nf

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        if self.s == 64 and self.in_dist:
            inputs = torch.from_numpy(self.reader['Sample_' + str(index + self.start)]["input"][:]).type(torch.float32).reshape(1, self.s, self.s)
            labels = torch.from_numpy(self.reader['Sample_' + str(index + self.start)]["output"][:]).type(torch.float32).reshape(1, self.s, self.s)

        else:

            inputs = self.reader['Sample_' + str(index + self.start)]["input"][:].reshape(1,1,self.s, self.s)
            labels = self.reader['Sample_' + str(index + self.start)]["output"][:].reshape(1,1, self.s, self.s)

            if self.s<128:
                inputs = downsample(inputs, self.s).reshape(1, self.s, self.s)
                labels = downsample(labels, self.s).reshape(1, self.s, self.s)
            else:
                inputs = inputs.reshape(1, 128, 128)
                labels = labels.reshape(1, 128, 128)

            inputs = torch.from_numpy(inputs).type(torch.float32)
            labels = torch.from_numpy(labels).type(torch.float32)

        inputs = (inputs - self.min_data)/(self.max_data - self.min_data)
        labels = (labels - self.min_model)/(self.max_model - self.min_model)


        if self.N_Fourier_F > 0:
            grid = self.get_grid()
            FF = FourierFeatures(1, self.N_Fourier_F, grid.device)
            ff_grid = FF(grid)
            ff_grid = ff_grid.permute(2, 0, 1)
            inputs = torch.cat((inputs, ff_grid), 0)

        return inputs, labels

    def get_grid(self):
        x = torch.linspace(0, 1, self.s)
        y = torch.linspace(0, 1, self.s)
        x_grid, y_grid = torch.meshgrid(x, y)
        x_grid = x_grid.unsqueeze(-1)
        y_grid = y_grid.unsqueeze(-1)
        grid = torch.cat((x_grid, y_grid), -1)
        return grid

class ShearLayer:
    def __init__(self, network_properties, device, batch_size, training_samples, size, in_dist = True):

        #Must have parameters: ------------------------------------------------

        if "in_size" in network_properties:
            self.in_size = network_properties["in_size"]
            assert self.in_size<=128
        else:
            raise ValueError("You must specify the computational grid size.")

        if "N_layers" in network_properties:
            N_layers = network_properties["N_layers"]
        else:
            raise ValueError("You must specify the number of (D) + (U) blocks.")

        if "N_res" in network_properties:
                N_res = network_properties["N_res"]
        else:
            raise ValueError("You must specify the number of (R) blocks.")

        if "N_res_neck" in network_properties:
                N_res_neck = network_properties["N_res_neck"]
        else:
            raise ValueError("You must specify the number of (R)-neck blocks.")

        #Load default parameters if they are not in network_properties
        network_properties = default_param(network_properties)

        #----------------------------------------------------------------------
        kernel_size = network_properties["kernel_size"]
        channel_multiplier = network_properties["channel_multiplier"]
        retrain = network_properties["retrain"]
        self.N_Fourier_F = network_properties["FourierF"]

        #Filter properties: ---------------------------------------------------
        cutoff_den = network_properties["cutoff_den"]
        filter_size = network_properties["filter_size"]
        half_width_mult = network_properties["half_width_mult"]
        lrelu_upsampling = network_properties["lrelu_upsampling"]
        activation = network_properties["activation"]
        ##----------------------------------------------------------------------

        torch.manual_seed(retrain)

        self.model = CNO(in_dim  = 1 + 2*self.N_Fourier_F,     # Number of input channels.
                        in_size = self.in_size,                # Input spatial size
                        N_layers = N_layers,                   # Number of (D) and (U) Blocks in the network
                        N_res = N_res,                         # Number of (R) Blocks per level
                        N_res_neck = N_res_neck,
                        channel_multiplier = channel_multiplier,
                        conv_kernel=kernel_size,
                        cutoff_den = cutoff_den,
                        filter_size=filter_size,
                        lrelu_upsampling = lrelu_upsampling,
                        half_width_mult  = half_width_mult,
                        activation = activation).to(device)

        #----------------------------------------------------------------------

        #Change number of workers accoirding to your preference

        num_workers = 0
        self.train_loader = DataLoader(ShearLayerDataset("training", self.N_Fourier_F, training_samples, size), batch_size=batch_size, shuffle=True, num_workers=num_workers)
        self.val_loader = DataLoader(ShearLayerDataset("validation", self.N_Fourier_F, training_samples, size), batch_size=batch_size, shuffle=False, num_workers=num_workers)
        self.test_loader = DataLoader(ShearLayerDataset("test", self.N_Fourier_F, training_samples, size, in_dist), batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [ ]:
example = ShearLayer(model_architecture_, device, batch_size, training_samples, size = 64)

In [ ]:
model = example.model
n_params = model.print_size()
train_loader = example.train_loader #TRAIN LOADER
val_loader = example.val_loader #VALIDATION LOADER

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
freq_print = 1

Total number of model parameters: 2951249 (~11.26MiB)


In [ ]:
if p == 1:
    loss = torch.nn.L1Loss()
elif p == 2:
    loss = torch.nn.MSELoss()

best_model_testing_error = 1000 #Save the model once it has less than 1000% relative L1 error
patience = int(0.2 * epochs)    # Early stopping parameter
counter = 0

In [ ]:
for epoch in range(epochs):
    with tqdm(unit="batch", disable=False) as tepoch:

        model.train()
        tepoch.set_description(f"Epoch {epoch}")
        train_mse = 0.0
        running_relative_train_mse = 0.0
        for step, (input_batch, output_batch) in enumerate(train_loader):
            optimizer.zero_grad()
            input_batch = input_batch.to(device)
            output_batch = output_batch.to(device)

            output_pred_batch = model(input_batch)

            loss_f = loss(output_pred_batch, output_batch) / loss(torch.zeros_like(output_batch).to(device), output_batch)

            loss_f.backward()
            optimizer.step()
            train_mse = train_mse * step / (step + 1) + loss_f.item() / (step + 1)
            tepoch.set_postfix({'Batch': step + 1, 'Train loss (in progress)': train_mse})

        writer.add_scalar("train_loss/train_loss", train_mse, epoch)

        with torch.no_grad():
            model.eval()
            test_relative_l2 = 0.0
            train_relative_l2 = 0.0

            for step, (input_batch, output_batch) in enumerate(val_loader):

                input_batch = input_batch.to(device)
                output_batch = output_batch.to(device)
                output_pred_batch = model(input_batch)

                loss_f = torch.mean(abs(output_pred_batch - output_batch)) / torch.mean(abs(output_batch)) * 100
                test_relative_l2 += loss_f.item()
            test_relative_l2 /= len(val_loader)

            for step, (input_batch, output_batch) in enumerate(train_loader):
                input_batch = input_batch.to(device)
                output_batch = output_batch.to(device)
                output_pred_batch = model(input_batch)
            train_relative_l2 /= len(train_loader)

            writer.add_scalar("train_loss/train_loss_rel", train_relative_l2, epoch)
            writer.add_scalar("val_loss/val_loss", test_relative_l2, epoch)

            if test_relative_l2 < best_model_testing_error:
                best_model_testing_error = test_relative_l2
                best_model = copy.deepcopy(model)
                torch.save(best_model, folder + "/model.pkl")
                writer.add_scalar("val_loss/Best Relative Testing Error", best_model_testing_error, epoch)
                counter = 0
            else:
                counter+=1

        tepoch.set_postfix({'Train loss': train_mse, "Relative Train": train_relative_l2, "Relative Val loss": test_relative_l2})
        tepoch.close()

        with open(folder + '/errors.txt', 'w') as file:
            file.write("Training Error: " + str(train_mse) + "\n")
            file.write("Best Testing Error: " + str(best_model_testing_error) + "\n")
            file.write("Current Epoch: " + str(epoch) + "\n")
            file.write("Params: " + str(n_params) + "\n")
        scheduler.step()

    if counter>patience:
        print("Early Stopping")
        break

Epoch 0: : 0batch [00:00, ?batch/s]

Setting up PyTorch plugin "filtered_lrelu_plugin"... 

/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


Done.


Epoch 0: : 0batch [02:37, ?batch/s, Train loss=0.307, Relative Train=0, Relative Val loss=29]
Epoch 1: : 0batch [00:01, ?batch/s, Train loss=0.131, Relative Train=0, Relative Val loss=17.8]
Epoch 2: : 0batch [00:02, ?batch/s, Train loss=0.122, Relative Train=0, Relative Val loss=15.6]
Epoch 3: : 0batch [00:02, ?batch/s, Train loss=0.119, Relative Train=0, Relative Val loss=13]
Epoch 4: : 0batch [00:01, ?batch/s, Train loss=0.123, Relative Train=0, Relative Val loss=12]
Epoch 5: : 0batch [00:01, ?batch/s, Train loss=0.13, Relative Train=0, Relative Val loss=13.4]
Epoch 6: : 0batch [00:01, ?batch/s, Train loss=0.117, Relative Train=0, Relative Val loss=18.1]
Epoch 7: : 0batch [00:01, ?batch/s, Train loss=0.113, Relative Train=0, Relative Val loss=12.2]
Epoch 8: : 0batch [00:01, ?batch/s, Train loss=0.112, Relative Train=0, Relative Val loss=12.9]
Epoch 9: : 0batch [00:01, ?batch/s, Train loss=0.114, Relative Train=0, Relative Val loss=12.9]
Epoch 10: : 0batch [00:02, ?batch/s, Train loss

Early Stopping


In [ ]:
rm -rf ~/.cache/torch_extensions/

In [ ]:
pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 9.0 MB/s eta 0:00:00
